# Задание 1 (4 балла)

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете `data/corpus_wsd_50k.txt`

В качестве метрики близости вы должны попробовать два подхода:

1) Jaccard score на множествах слов (определений и контекста)
2) Cosine distance на эмбедингах sentence_transformers

В качестве метрики используйте accuracy (% правильных ответов). Предсказывайте только многозначные слова в датасете

Контекст вы можете определить самостоятельно (окно вокруг целевого слова или все предложение). Также можете поэкспериментировать с предобработкой для обоих методов.

## Решение:

### Загрузка датасета


In [3]:
import pathlib

corpus_path = pathlib.Path(r"data/corpus_wsd_50k.txt")

with open(corpus_path, "r", encoding="utf-8") as file:
    corpus = file.read().split('\n\n')
    corpus = map(
        lambda sent: [
            word.split("\t")
            for word
            in sent.split("\n")
            ],
        corpus
        )
    corpus = tuple(corpus)

###  Загрузка ворднета

In [4]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
    

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kirill\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


###  Функция, составляющая контекст

Конетекстом считаем предложение, в котором встречается слово, без самого слова. Будем брать предложение из датасета целиком и маскировать в нем слово, для которого нужно определить значение. 

In [5]:
def get_context_mask_target(corpus_sentence:list, target_ind:int) -> list:

    result = list()

    for ind, word in enumerate(corpus_sentence):
        if ind != target_ind:
            result.append(word[1])
        else:
            result.append("<mask/>")

    return result

### Препроц

In [6]:
from string import punctuation


class Preprocessor:

    stopwords:list[str] = nltk.corpus.stopwords.words("english")
    stopwords.extend(punctuation)
    stopwords = set(stopwords)

    @classmethod
    def tokenize(cls, text:str) -> list[str]:
        return nltk.word_tokenize(text)
    
    @classmethod
    def rm_stopwords(cls, tokenized_text:list[str]) -> list[str]:
        return [c for c in tokenized_text if c not in cls.stopwords]

    @classmethod
    def preprocess(cls, text: str | list[str]) -> list[str]:
        
        if isinstance(text, str):
            text = cls.tokenize(text)
        
        text = [c.lower() for c in text]      
        text = cls.rm_stopwords(text)

        return text

In [7]:
#  Проверяем препроц

s = corpus[1]
s = get_context_mask_target(s, 8)
s = Preprocessor.preprocess(s)
s

['permit',
 'become',
 'giveaway',
 '<mask/>',
 'rather',
 'one',
 'goal',
 'improved',
 'employee',
 'morale',
 'consequently',
 'increased',
 'productivity']

###  Алгоритм Леска в общем виде

In [33]:
class Lesk:

    #  Псевдометрика, переопределяется в классах-наследниках
    metric = lambda sent1, sent2: 0

    @classmethod
    def __get_all_definitions(cls, word:str) -> str:
        return [c.definition() for c in wn.synsets(word)]


    @classmethod
    def get_definition(cls, word:str, context:list[str]) -> str:

        all_results = [
            {
                "definition": definition,
                "score"     : cls.metric(context, Preprocessor.preprocess(definition))
            }
            for definition
            in cls.__get_all_definitions(word)
        ]

        return max(all_results, key=lambda c: c["score"])


### Алгоритм Леска с метрикой Жаккара

In [34]:
def jaccard(sent1:list[str], sent2:list[str]) -> float:
    sent1 = set(sent1)
    sent2 = set(sent2)

    return len(sent1 & sent2) / len(sent1 | sent2)


class LeskWithJaccard(Lesk):

    metric = jaccard

In [35]:
#  Проверка:

word = "car"
ctx = "I drive my car with my personnel in it to my dads power plant."
ctx = Preprocessor.preprocess(ctx)

LeskWithJaccard.get_definition(word, ctx)

{'definition': 'the compartment that is suspended from an airship and that carries personnel and the cargo and the power plant',
 'score': 0.2727272727272727}

In [42]:
word = "car"
ctx = "I have just bought a new BMW car from the shop and it's amazing"
ctx = Preprocessor.preprocess(ctx)

LeskWithJaccard.get_definition(word, ctx)

{'definition': 'a motor vehicle with four wheels; usually propelled by an internal combustion engine',
 'score': 0.0}

### ### Алгоритм Леска с косинусной близостью на эмбедингах sentence_transformers

In [36]:
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sentence_transformers import SentenceTransformer

In [37]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

d:\HSE subj data\актуальные проблемы компьютерной лингвистики\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [38]:

def cos_sim(sent1:list[str], sent2:list[str]) -> float:
    emb1 = model.encode(" ".join(sent1)).reshape(1, -1)
    emb2 = model.encode(" ".join(sent2)).reshape(1, -1)

    return float(cosine_distances(emb1, emb2)[0][0])

class LeskWithCosSim(Lesk):

    metric = cos_sim

In [39]:
#  Проверка:

word = "car"
ctx = "I drive my car with my personnel in it to my dads power plant."
ctx = Preprocessor.preprocess(ctx)

LeskWithCosSim.get_definition(word, ctx)

{'definition': 'where passengers ride up and down',
 'score': 0.8298293352127075}

In [40]:
word = "car"
ctx = "I have just bought a new BMW car from the shop and it's amazing"
ctx = Preprocessor.preprocess(ctx)

LeskWithCosSim.get_definition(word, ctx)

{'definition': 'a wheeled vehicle adapted to the rails of railroad',
 'score': 0.932273805141449}

# Задание 2 (4 балла)
Попробуйте разные алгоритмы кластеризации на датасете - `https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv`

Используйте код из семинара как основу. Используйте ARI как метрику качества.

Попробуйте все 4 алгоритма кластеризации, про которые говорилось на семинаре. Для каждого из алгоритмов попробуйте настраивать гиперпараметры (посмотрите их в документации). Прогоните как минимум 5 экспериментов (не обязательно успешных) с разными параметрами на каждый алгоритме кластеризации и оцените: качество кластеризации, скорость работы, интуитивность параметров.

Помимо этого также выберите 1 дополнительный алгоритм кластеризации отсюда - https://scikit-learn.org/stable/modules/clustering.html , опишите своими словами принцип его работы  и проделайте аналогичные эксперименты. 